In [36]:
import cv2
import numpy as np
import os
import time
from pyduino import Arduino
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [58]:

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

face_data = []
labels = []

class_id = 0
names = {}

#data Preparation
for fx in os.listdir():
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        data_item = np.load(fx)
        face_data.append(data_item)

        #create lables for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id +=1
        labels.append(target)
        
X_train = face_data
labels = []
for i in range(0,len(X_train)):
    labels.append(i)
labels = np.array(labels)
recognizer.train(X_train, labels)
#recognizer.save('trainner.yml')

In [59]:
# Checking the Arduino 
try:
    ard = Arduino('COM11',9600)
    
    print('Connection established')
    ard.close()
except:
    print('Error!! Unable to establish connection with Arduino!')
    

Error!! Unable to establish connection with Arduino!


In [60]:
def go_arduino():
    try:
        arduino = Arduino('COM11',9600)
        arduino.knock()
        arduino.close()
    except:
        print('Error in connecting ')

In [61]:
cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h = face

        #Get the ROI
        offset = 10
        face_section = gray[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        out, conf = recognizer.predict(face_section)
        if conf>45 and conf<100:
            print(out)
            pred_name = names[out]
            cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
            #if(pred_name == 'Prashant'):
                #go_arduino()
        else:
            cv2.putText(frame,'UnKnown Face',(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [52]:
cap.release()
cv2.destroyAllWindows()

In [9]:
go_arduino()

Error in connecting 
